In [1]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

/tmp/ipykernel_1252657/658913012.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
model_path = "gpt2-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt2-small/checkpoint-trainedtokenizer_100M_whitespace"
#model_path = "gpt3-small/checkpoint-trainedtokenizer_10M_whitespace"
#model_path = "gpt3-small/checkpoint-trainedtokenizer_100M_whitespace"

model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

# wrap with minicons scorer
lm_scorer = scorer.IncrementalLMScorer(model_path, device = "cpu")

In [3]:
def calculate_surprisal(sentence):
    '''
    Takes in a sentence, and outputs surprisal values for each word.
    '''
    
    input_sentence = sentence
    # token_score() function of Minicons takes in several parameters
    # if surprisal = True, the output value is surprisal instead of log likelihood
    # if base_two = True, the log likelihood will be in base 2
    # see Minicons documentations for details
    # score tokens
    token_surprisals = lm_scorer.token_score(input_sentence, surprisal = True, base_two = True)[0]
    #print(token_surprisals)

    '''
    # filter out special tokens (like <pad>)
    special_tokens = set(tokenizer.all_special_tokens + ['<pad>'])
    filtered = [
        (token, score)
        for (token, score) in token_surprisals
        if token not in special_tokens
    ]
    '''

   # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    
    #print(expanded)

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results


sentence = 'I know that your friend gave a baguette to Mary last weekend.'
calculate_surprisal(sentence)

[('I', 0.0),
 ('know', 5.239012718200684),
 ('that', 3.954333782196045),
 ('your', 10.374737739562988),
 ('friend', 6.853007793426514),
 ('gave', 12.485967636108398),
 ('a', 4.920528888702393),
 ('baguette', 40.79525184631348),
 ('to', 8.777816772460938),
 ('Mary', 12.327611923217773),
 ('last', 11.225496292114258),
 ('weekend', 8.15101146697998),
 ('.', 10.069524765014648)]

In [4]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(relevant_surprisals)

In [5]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df
    
def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part).strip() for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

def assign_grammaticality(row):
    ''' Determines grammaticality based on licensor and gap values.'''
    if row['filler'] > 0 and row['gap'] == 1:
        return 'gram+'
        #return 1
    elif row['filler'] == 0 and row['gap'] == 0:
        return 'gram+'
        #return 1
    elif row['filler'] > 0 and row['gap'] == 0:
        return 'gram-'
        #return 0
    elif row['filler'] == 0 and row['gap'] == 1:
        return 'gram-'
        #return 0

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [6]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [7]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
        
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [8]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

In [9]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality, axis = 1)

Gap Position == Object

In [10]:
# modifier: short_mod, med_mod, long_mod

In [11]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [12]:
# flip test: gap-
no_gap_df = objectgap_df[objectgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'object gap position: gap-')


=== OBJECT GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 1.3652
Min. group size: 8
Log-Likelihood: -307.7916
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,33.173,1.579,21.015,0.000,30.079,36.267
1,wh_numeric,0.110,0.092,1.191,0.234,-0.071,0.291



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,49.667,14.747


In [13]:
# flip test: gap+
gap_df = objectgap_df[objectgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'object gap position: gap+')


=== OBJECT GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 1.5233
Min. group size: 8
Log-Likelihood: -319.2730
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,29.858,1.935,15.434,0.000,26.067,33.650
1,wh_numeric,0.056,0.098,0.572,0.567,-0.135,0.247



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,74.662,20.978


Gap Postion == PP/Goal

In [15]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [16]:
# flip test: gap-
no_gap_df = ppgap_df[ppgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'pp gap position: gap-')


=== PP GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.4557
Min. group size: 8
Log-Likelihood: -235.3753
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,23.351,1.932,12.087,0.000,19.564,27.138
1,wh_numeric,-0.056,0.053,-1.051,0.293,-0.161,0.049



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,74.593,38.245


In [17]:
# flip test: gap+
gap_df = ppgap_df[ppgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'pp gap position: gap+')


=== PP GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.8238
Min. group size: 8
Log-Likelihood: -283.4059
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,30.313,2.776,10.922,0.000,24.873,35.753
1,wh_numeric,-0.087,0.072,-1.210,0.226,-0.227,0.054



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,153.970,58.702


Embedded Clause

In [10]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [11]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [12]:
# flip test: gap-
no_gap_df = embed_objectgap_df[embed_objectgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'object gap position: gap-')


=== OBJECT GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.3108
Min. group size: 8
Log-Likelihood: -163.7791
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.265,0.181,62.232,0.000,10.911,11.620
1,wh_numeric,-0.005,0.044,-0.123,0.902,-0.092,0.081



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.617,0.407


In [13]:
# flip test: gap+
gap_df = embed_objectgap_df[embed_objectgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'object gap position: gap+')


=== OBJECT GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.4627
Min. group size: 8
Log-Likelihood: -195.4126
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.052,0.223,49.556,0.000,10.615,11.489
1,wh_numeric,-0.019,0.054,-0.349,0.727,-0.124,0.087



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.937,0.506


In [14]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'object gap position: division by grammaticality')


=== OBJECT GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.4132
Min. group size: 16
Log-Likelihood: -348.2115
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.152,0.201,55.405,0.000,10.758,11.547
1,grammaticality[T.gram-],0.013,0.072,0.186,0.853,-0.128,0.154



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.759,0.408


Embedded Clause - PP/Goal Gap Position

In [15]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [16]:
# flip test: gap-
no_gap_df = embed_ppgap_df[embed_ppgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'pp gap position: gap-')


=== PP GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.3055
Min. group size: 8
Log-Likelihood: -162.0638
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.299,0.176,64.187,0.000,10.954,11.644
1,wh_numeric,0.028,0.044,0.637,0.524,-0.058,0.113



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.582,0.388


In [17]:
# flip test: gap+
gap_df = embed_ppgap_df[embed_ppgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'pp gap position: gap+')


=== PP GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.3695
Min. group size: 8
Log-Likelihood: -172.1577
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.559,0.149,77.395,0.000,11.267,11.852
1,wh_numeric,0.037,0.048,0.777,0.437,-0.057,0.132



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.400,0.254


In [18]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'pp gap position: division by grammaticality')


=== PP GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.3749
Min. group size: 16
Log-Likelihood: -329.2151
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.434,0.161,71.237,0.000,11.119,11.748
1,grammaticality[T.gram-],-0.010,0.068,-0.139,0.889,-0.144,0.125



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.468,0.269


#### Double Gaps

In [19]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [20]:
def assign_grammaticality_double_gaps(row):
    ''' Determines grammaticality based on licensor and gap values for the double gap construction.'''
    if row['filler'] == 1 and row['gap_count'] == 1:
        return 'gram+'
    elif row['filler'] == 0 and row['gap_count'] == 0:
        return 'gram+'
    elif row['filler'] == 1 and row['gap_count'] == 0:
        return 'gram-'
    elif row['filler'] == 0 and row['gap_count'] == 1:
        return 'gram-'
    elif row['filler'] == 0 and row['gap_count'] == 2:
        return 'gram-'
    elif row['filler'] == 1 and row['gap_count'] == 2:
        return 'gram-'

In [21]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_count'] = sentence_df['subject_gap'] + sentence_df['object_gap']
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality_double_gaps, axis = 1)

Embedded Clause

In [22]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [23]:
# flip test: gap = 0
gap_0 = sentence_df[sentence_df['gap_count'] == 0]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_0, iv_list, 'embed_surprisal', 'double gap embed: gap = 0')


=== DOUBLE GAP EMBED: GAP = 0 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 40
Method: REML
No. Groups: 20
Scale: 0.0190
Min. group size: 2
Log-Likelihood: -23.2126
Max. group size: 2
Converged: Yes
Mean group size: 2.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,9.223,0.187,49.191,0.000,8.855,9.590
1,wh_numeric,-0.013,0.022,-0.608,0.543,-0.056,0.029



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.694,2.338


In [24]:
# flip test: gap = 1
gap_1 = sentence_df[sentence_df['gap_count'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_1, iv_list, 'embed_surprisal', 'double gap embed: gap = 1')


=== DOUBLE GAP EMBED: GAP = 1 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 80
Method: REML
No. Groups: 20
Scale: 0.2985
Min. group size: 4
Log-Likelihood: -90.0270
Max. group size: 4
Converged: Yes
Mean group size: 4.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,9.702,0.196,49.597,0.000,9.319,10.086
1,wh_numeric,0.029,0.061,0.470,0.639,-0.091,0.148



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.691,0.523


In [25]:
# flip test: gap = 2
gap_2 = sentence_df[sentence_df['gap_count'] == 2]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_2, iv_list, 'embed_surprisal', 'double gap embed: gap = 2')


=== DOUBLE GAP EMBED: GAP = 2 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 40
Method: REML
No. Groups: 20
Scale: 0.0487
Min. group size: 2
Log-Likelihood: -40.2376
Max. group size: 2
Converged: Yes
Mean group size: 2.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,10.083,0.287,35.124,0.000,9.520,10.645
1,wh_numeric,0.076,0.035,2.175,0.030,0.008,0.144



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.624,3.425


In [26]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed: division by grammaticality')


=== DOUBLE GAP EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.3944
Min. group size: 8
Log-Likelihood: -181.8138
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,9.566,0.213,44.885,0.000,9.148,9.984
1,grammaticality[T.gram-],0.178,0.103,1.740,0.082,-0.023,0.379



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.777,0.455


### Islands

In [27]:
def assign_grammaticality_islands(row, island_type):
    ''' Determines grammaticality based on licensor and gap values.'''
    if row[island_type] == 'null' or row[island_type] == 'that':
        if row['filler'] > 0 and row['gap'] == 1:
            return 'gram+'
            #return 1
        elif row['filler'] == 0 and row['gap'] == 0:
            return 'gram+'
            #return 1
        elif row['filler'] > 0 and row['gap'] == 0:
            return 'gram-'
            #return 0
        elif row['filler'] == 0 and row['gap'] == 1:
            return 'gram-'
            #return 0
    else:
        if row['filler'] > 0 and row['gap'] == 1:
            return 'gram-'
            #return 0
        elif row['filler'] == 0 and row['gap'] == 0:
            return 'gram+'
            #return 1
        elif row['filler'] > 0 and row['gap'] == 0:
            return 'gram-'
            #return 0
        elif row['filler'] == 0 and row['gap'] == 1:
            return 'gram-'
            #return 0

#### Wh-Islands

In [28]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [29]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)
sentence_df['grammaticality'] = sentence_df.apply(lambda x: assign_grammaticality_islands(x, 'comp_type'), axis = 1)

Post-Gap Region - Continuation

In [10]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [11]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'wh-islands continuation: gap-')


=== WH-ISLANDS CONTINUATION: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.2581
Min. group size: 6
Log-Likelihood: -161.0382
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,21.455,1.452,14.779,0.000,18.610,24.301
1,comp_type[T.that],-0.226,0.114,-1.991,0.047,-0.449,-0.003
2,comp_type[T.whether],-0.097,0.114,-0.853,0.394,-0.320,0.126
3,wh_numeric,0.101,0.080,1.261,0.207,-0.056,0.259
4,wh_numeric:comp_type[T.that],-0.298,0.114,-2.626,0.009,-0.521,-0.076
5,wh_numeric:comp_type[T.whether],-0.346,0.114,-3.049,0.002,-0.569,-0.124



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,42.023,29.427


In [12]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'wh-islands continuation: gap+')


=== WH-ISLANDS CONTINUATION: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.3763
Min. group size: 6
Log-Likelihood: -184.4054
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,31.405,1.935,16.227,0.000,27.612,35.198
1,comp_type[T.that],0.013,0.137,0.091,0.927,-0.256,0.281
2,comp_type[T.whether],-0.203,0.137,-1.477,0.140,-0.471,0.066
3,wh_numeric,-0.098,0.097,-1.012,0.312,-0.288,0.092
4,wh_numeric:comp_type[T.that],0.182,0.137,1.328,0.184,-0.087,0.451
5,wh_numeric:comp_type[T.whether],0.166,0.137,1.213,0.225,-0.102,0.435



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,74.721,43.329


Embedded Clause

In [30]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [31]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'wh-islands embed: gap-')


=== WH-ISLANDS EMBED: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0263
Min. group size: 6
Log-Likelihood: -20.5380
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.131,0.269,41.429,0.000,10.605,11.658
1,comp_type[T.that],-0.231,0.036,-6.352,0.000,-0.302,-0.159
2,comp_type[T.whether],-0.072,0.036,-1.979,0.048,-0.143,-0.001
3,wh_numeric,0.165,0.026,6.410,0.000,0.114,0.215
4,wh_numeric:comp_type[T.that],-0.052,0.036,-1.431,0.152,-0.123,0.019
5,wh_numeric:comp_type[T.whether],-0.077,0.036,-2.110,0.035,-0.148,-0.005



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.431,3.143


In [32]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'wh-islands embed: gap+')


=== WH-ISLANDS EMBED: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0374
Min. group size: 6
Log-Likelihood: -38.9122
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.631,0.294,39.539,0.000,11.054,12.207
1,comp_type[T.that],-0.341,0.043,-7.894,0.000,-0.426,-0.257
2,comp_type[T.whether],-0.181,0.043,-4.188,0.000,-0.266,-0.096
3,wh_numeric,0.147,0.031,4.791,0.000,0.087,0.206
4,wh_numeric:comp_type[T.that],0.001,0.043,0.031,0.975,-0.083,0.086
5,wh_numeric:comp_type[T.whether],-0.032,0.043,-0.748,0.454,-0.117,0.052



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.712,3.156


In [33]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed: division by grammaticality')


=== WH-ISLANDS EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.1388
Min. group size: 12
Log-Likelihood: -154.0293
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.188,0.280,40.002,0.000,10.640,11.736
1,grammaticality[T.gram-],0.095,0.049,1.945,0.052,-0.001,0.191



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.537,1.405


#### Adjunct Islands 

In [34]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [35]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [36]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)
sentence_df['grammaticality'] = sentence_df.apply(lambda x: assign_grammaticality_islands(x, 'adjunct_type'), axis = 1)

Post-Gap Region - Continuation

In [21]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [22]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'adjunct islands continuation: gap-')


=== ADJUNCT ISLANDS CONTINUATION: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.4314
Min. group size: 6
Log-Likelihood: -194.4080
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,32.665,2.327,14.037,0.000,28.104,37.226
1,adjunct_type[T.back],-0.059,0.147,-0.400,0.689,-0.347,0.229
2,adjunct_type[T.front],0.110,0.147,0.752,0.452,-0.177,0.398
3,wh_numeric,0.081,0.104,0.784,0.433,-0.122,0.285
4,wh_numeric:adjunct_type[T.back],-0.052,0.147,-0.355,0.722,-0.340,0.236
5,wh_numeric:adjunct_type[T.front],-0.049,0.147,-0.334,0.738,-0.337,0.239



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,108.092,58.520


In [23]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'adjunct islands continuation: gap+')


=== ADJUNCT ISLANDS CONTINUATION: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.4485
Min. group size: 6
Log-Likelihood: -192.4646
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,42.796,1.907,22.439,0.000,39.058,46.534
1,adjunct_type[T.back],-0.042,0.150,-0.282,0.778,-0.336,0.251
2,adjunct_type[T.front],0.063,0.150,0.419,0.675,-0.231,0.356
3,wh_numeric,0.010,0.106,0.093,0.926,-0.198,0.217
4,wh_numeric:adjunct_type[T.back],-0.007,0.150,-0.049,0.961,-0.301,0.286
5,wh_numeric:adjunct_type[T.front],0.061,0.150,0.406,0.685,-0.233,0.354



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,72.522,38.527


Embedded Clause

In [37]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [38]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'adjunct islands embed: gap-')


=== ADJUNCT ISLANDS EMBED: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0961
Min. group size: 6
Log-Likelihood: -82.3969
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.119,0.276,40.224,0.000,10.577,11.661
1,adjunct_type[T.back],0.221,0.069,3.194,0.001,0.085,0.357
2,adjunct_type[T.front],0.301,0.069,4.337,0.000,0.165,0.436
3,wh_numeric,0.018,0.049,0.368,0.713,-0.078,0.114
4,wh_numeric:adjunct_type[T.back],0.019,0.069,0.281,0.779,-0.116,0.155
5,wh_numeric:adjunct_type[T.front],0.105,0.069,1.518,0.129,-0.031,0.241



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.480,1.716


In [39]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'adjunct islands embed: gap+')


=== ADJUNCT ISLANDS EMBED: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1145
Min. group size: 6
Log-Likelihood: -91.0274
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,10.791,0.281,38.356,0.000,10.239,11.342
1,adjunct_type[T.back],0.388,0.076,5.126,0.000,0.240,0.536
2,adjunct_type[T.front],0.487,0.076,6.436,0.000,0.339,0.635
3,wh_numeric,0.013,0.053,0.250,0.803,-0.091,0.118
4,wh_numeric:adjunct_type[T.back],0.024,0.076,0.319,0.750,-0.124,0.172
5,wh_numeric:adjunct_type[T.front],0.122,0.076,1.615,0.106,-0.026,0.270



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.526,1.623


In [40]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed: division by grammaticality')


=== ADJUNCT ISLANDS EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.2357
Min. group size: 12
Log-Likelihood: -211.1722
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,11.126,0.271,41.057,0.000,10.595,11.657
1,grammaticality[T.gram-],0.092,0.066,1.390,0.164,-0.038,0.223



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,1.410,0.996


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality_islands, axis = 1)

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
# flip test: gap-
no_gap_df = subject_df[subject_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'complex np islands subject condition: gap-')

In [ ]:
# flip test: gap+
gap_df = subject_df[subject_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'complex np islands subject condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands subject condition: division by grammaticality')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
# flip test: gap-
no_gap_df = object_df[object_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'complex np islands object condition: gap-')

In [ ]:
# flip test: gap+
gap_df = object_df[object_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'complex np islands object condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands object condition: division by grammaticality')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
# flip test: gap-
no_gap_df = subject_df[subject_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: gap-')

In [ ]:
# flip test: gap+
gap_df = subject_df[subject_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: division by grammaticality')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
# flip test: gap-
no_gap_df = object_df[object_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'complex np islands object condition: gap-')

In [ ]:
# flip test: gap+
gap_df = object_df[object_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'complex np islands object condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands object condition: division by grammaticality')